In [1]:
#!pip install -r requirements.txt --user
import os
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np
import seaborn as sns
from datetime import datetime
import math
import unicodedata
from unicodedata import normalize
import re
from dateutil.relativedelta import relativedelta
import pingouin
import sklearn
#from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
import scipy
from scipy.stats import chi2_contingency

## 0.0 Arreglando las rutas ##

In [2]:
ruta_madre="C:/Users/USUARIO/Documents/GitHub/Team_224/Proyecto"
ruta_insumos=os.path.join(ruta_madre, 'Insumos')
ruta_resultados=os.path.join(ruta_madre, 'Resultados')

## 1.0 IMPORTANDO LA DATA ##


### 1.1 Importando sueldos (info_nomina)


In [3]:
os.chdir(os.path.join(ruta_insumos,'Descripcion_empleados'))
sendero_sueldos=os.path.join(os.getcwd(),'Sueldos_nomina_consolidados.csv')
sueldos=pd.read_csv(sendero_sueldos,sep=";")
#sueldos['year']=sueldos['year'].astype('float64')
sueldos['Codigo']=sueldos['Codigo'].astype('int').astype('str')
sueldos.drop_duplicates(['year','Codigo','Nombres'],inplace=True)

### 1.3 Importando datos demograficos (perfil_socidemografico)

In [4]:
os.chdir(os.path.join(ruta_insumos,"Encuesta_sociodem_empleados"))
sendero_feature=os.path.join(os.getcwd(),str(os.listdir()[0]))
feature_demo=pd.read_excel(sendero_feature)
#feature_demo.dropna(subset=["NOMBRE_CLAVE"],inplace=True)

#estadisticos_iniciales=consol_data.describe(include = ['O'])


### 1.4 Importando datos de actividades de solicitudes (detalle_tramites)

In [6]:
os.chdir(os.path.join(ruta_insumos,'Datos_eventos'))
sendero_eventos=os.path.join(os.getcwd(),str(os.listdir()[0]))
datos_eventos=pd.read_excel(sendero_eventos)
datos_eventos['cedula']=datos_eventos['cedula'].astype('str')
#datos_eventos.dropna(subset=["NOMBRE_CLAVE"],inplace=True)


## 2.0 LIMPIANDO LA DATA Y PRIOR FEATURE ENGINEERING ##

In [7]:

feature_demo['NUMERO_DE_DOCUMENTO']=feature_demo.NUMERO_DE_DOCUMENTO.astype('int').astype('str').str.strip()
sueldos['Codigo']=sueldos['Codigo'].str.strip()
consol_data_con_features=datos_eventos.merge(feature_demo,how='left',right_on='NUMERO_DE_DOCUMENTO',left_on='cedula')

data_agregada=consol_data_con_features.merge(sueldos[['year','Codigo','Sueldo','Cargo']],how='left',left_on=['year','cedula'],right_on=['year','Codigo'])


### 2.1 creando la edad y años de experiencia

In [8]:
data_agregada['fecha_analisis']=[pd.to_datetime(datetime(data_agregada['year'][x],data_agregada['MES'][x],1)) for x in range(len(data_agregada['year']))]

years_born=data_agregada['fecha_analisis']-data_agregada['FECHA_DE_NACIMIENTO']

data_agregada['edad']=years_born.dt.days//360; del years_born

expertise=data_agregada['fecha_analisis']-data_agregada['FECHA_DE_INGRESO']
data_agregada['years_antiguedad']=expertise.dt.days/360 ; del expertise

data_agregada.drop(columns=['NOMBRES','APELLIDOS','Codigo','NUMERO_DE_DOCUMENTO'],inplace=True)


### 2.2 Arreglando variable 'MUNICIPIO_DE_RESIDENCIA'

In [9]:
data_agregada['MUNICIPIO_DE_RESIDENCIA']=data_agregada.MUNICIPIO_DE_RESIDENCIA.str.lower().str.strip().str.replace('[ ]+','_')

def normalizar(x):
    
    caracter_norm=re.sub(
        r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
            normalize( "NFD", x
            ), 0, re.I
        )
    return caracter_norm

prueba=[]
for i in range(len(data_agregada['MUNICIPIO_DE_RESIDENCIA'])):
    
    try:
        prueba.append(normalizar(data_agregada['MUNICIPIO_DE_RESIDENCIA'][i]))
    except:
        
        prueba.append(np.nan)
    
    
data_agregada['MUNICIPIO_DE_RESIDENCIA']=prueba

C:\Users\USUARIO\AppData\Local\Temp/ipykernel_7548/1830578000.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data_agregada['MUNICIPIO_DE_RESIDENCIA']=data_agregada.MUNICIPIO_DE_RESIDENCIA.str.lower().str.strip().str.replace('[ ]+','_')


### 2.3 Normalizando y estandarizando solicitudes

In [12]:
## que quede solicitudes, autorizaciones por dia para evitar el sesgo por mayores dias en un mes

data_agregada['fecha_fin']=pd.to_datetime([data_agregada['fecha_analisis'][j]+relativedelta(months=1) for j in range(len(data_agregada['fecha_analisis']))])
data_agregada['dias_mes']=data_agregada['fecha_fin']-data_agregada['fecha_analisis']
data_agregada['dias_mes']=data_agregada['dias_mes'].dt.days
data_agregada['solicitudes_per_day']=data_agregada['Solicitud']/data_agregada['dias_mes']
data_agregada['autorizaciones_per_day']=data_agregada['Autorizacion']/data_agregada['dias_mes']
data_agregada['codigos_per_day']=data_agregada['codigo']/data_agregada['dias_mes']



### 2.4 Normalizando arreglando variable 'PROFESION'

In [13]:
data_agregada['PROFESION']=data_agregada['PROFESION'].str.lower()

prueba=[]
for i in range(len(data_agregada['PROFESION'])):
    
    try:
        prueba.append(normalizar(data_agregada['PROFESION'][i]))
    except:
        
        prueba.append(np.nan)
   
    
data_agregada['PROFESION']=prueba

profesiones={'administracion de empresas':'admin_empresas',
             'administrador en servicios de la salud':'admin_servicios_salud',
             'secretaria':'secretariado',
             'asistente administrativo':'asistente_administrativo',
             'tecnico en auxiliar de enfermeria':'auxiliar_enfermeria',
             'auxiliar de enfermeria':'auxiliar_enfermeria',
             'tecnico auxiliar de enfermera':'auxiliar_enfermeria',
             'tecnico auxiliar de enfemeria':'auxiliar_enfermeria',
             'tecnico en agente  call center':'agente_call_center',
             'tecnico agente de contact center':'agente_call_center',
             'administrador en salud':'admin_servicios_salud',
             'administracion en salud':'admin_servicios_salud',
             'administrador de empresas':'admin_empresas',
             'tecnico en asistencia administrativa':'asistente_administrativo'
             }

data_agregada['PROFESION']=data_agregada['PROFESION'].replace(profesiones)


### 2.4 Normalizando arreglando variable 'PROFESION'

In [14]:
prueba=[]
for i in range(len(data_agregada['NIVEL_DE_ESCOLARIDAD'])):
    
    try:
        prueba.append(normalizar(data_agregada['NIVEL_DE_ESCOLARIDAD'][i]))
    except:
        
        prueba.append(np.nan)
   
  
data_agregada['NIVEL_DE_ESCOLARIDAD']=prueba


## 3.0 Exploratory Data Analysis ##

Información general de la tasa  consolidada

In [16]:
data_agregada.info().to_frame() ## la variables provenientes de los sueldos año a año, tienen cerca del 56% de la data dañada por falta de info de sueldos en ciertos años, 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1683 entries, 0 to 1682
Data columns (total 38 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   year                         1683 non-null   int64         
 1   MES                          1683 non-null   int64         
 2   NOMBRE_USUARIO               1683 non-null   object        
 3   cedula                       1683 non-null   object        
 4   Solicitud                    1683 non-null   int64         
 5   Autorizacion                 1683 non-null   int64         
 6   codigo                       1683 non-null   int64         
 7   DOCUMENTO_DE_IDENTIFICACION  1680 non-null   object        
 8   GENERO                       1680 non-null   object        
 9   FECHA_DE_NACIMIENTO          1680 non-null   datetime64[ns]
 10  LUGAR_DE_NACIMIENTO          1680 non-null   object        
 11  POBLACION_ESPECIAL           1680 non-null 

AttributeError: 'NoneType' object has no attribute 'to_frame'

In [ ]:


## describe las variables numericas y categoricas
data_agg_describe_numeric=data_agregada.describe()
data_agg_describe_object=data_agregada.describe(include=['O'])